In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split

2023-04-20 01:02:34.192919: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
colour_ints = {"blue_cone": 0,
               "yellow_cone": 1,
               "orange_cone": 2,
               "large_orange_cone": 3
               }

In [3]:
pics = "/Users/stephenmesser/Desktop/FS-AI/extracted_cones"
directory = os.fsdecode(pics)

all_imgs = []
all_labels = []

for file_name in os.listdir(directory):

  if 'cone' in file_name and 'unknown' not in file_name:

    # add image to array
    img = cv2.imread(f'{pics}/{file_name}')
    res_image = cv2.resize(img, dsize=(25, 25))
    all_imgs.append(res_image)

    # add label to array
    all_labels.append(colour_ints[file_name.split('_cone_')[0] + '_cone'])

all_imgs = np.array(all_imgs)
print(all_imgs.shape)

(26508, 25, 25, 3)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.subplot(2,2,1)
plt.imshow(all_imgs[0], cmap=plt.get_cmap('gray'))
plt.subplot(2,2,2)
plt.imshow(all_imgs[1], cmap=plt.get_cmap('gray'))
plt.subplot(2,2,3)
plt.imshow(all_imgs[2], cmap=plt.get_cmap('gray'))
plt.subplot(2,2,4)
plt.imshow(all_imgs[61], cmap=plt.get_cmap('gray'))

#show the plot
plt.show()

In [4]:
#print(all_imgs[0])
# SCALING
all_imgs = all_imgs / 255
#print(all_imgs[0])

# ONE-HOT ENCODING
all_labels = np_utils.to_categorical(all_labels)
#print(all_labels)

In [5]:
num_classes = all_labels.shape[1]

In [6]:
reviews_train, reviews_test, y_train, y_test = train_test_split(all_imgs, all_labels, test_size=0.20)

In [7]:
len(reviews_train)

21206

In [16]:
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(25, 25, 3), activation='relu'))
    model.add(MaxPooling2D(strides=(5, 5)))
    model.add(MaxPooling2D(strides=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    #Compile/train model
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model


In [17]:
#define the model
model = cnn_model()
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 21, 21, 32)        2432      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 4, 4, 32)         0         
 2D)                                                             
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 2, 2, 32)         0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 2, 2, 32)          0         
                                                                 
 flatten_4 (Flatten)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [18]:
#train the model
training = model.fit(reviews_train, y_train, validation_split = 0.2, epochs=6, batch_size=200)
#Evaluation of the model
scores = model.evaluate(reviews_test, y_test, verbose=0)
print("CNN Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/6
85/85 [==============================] - 3s 32ms/step - loss: 1.2742 - accuracy: 0.4824 - val_loss: 1.1982 - val_accuracy: 0.6900
Epoch 2/6
85/85 [==============================] - 2s 28ms/step - loss: 1.1580 - accuracy: 0.6353 - val_loss: 1.0530 - val_accuracy: 0.6985
Epoch 3/6
85/85 [==============================] - 2s 28ms/step - loss: 0.9989 - accuracy: 0.6786 - val_loss: 0.8746 - val_accuracy: 0.6947
Epoch 4/6
85/85 [==============================] - 2s 29ms/step - loss: 0.8430 - accuracy: 0.6955 - val_loss: 0.7322 - val_accuracy: 0.7178
Epoch 5/6
85/85 [==============================] - 2s 29ms/step - loss: 0.7254 - accuracy: 0.7295 - val_loss: 0.6316 - val_accuracy: 0.7610
Epoch 6/6
85/85 [==============================] - 2s 28ms/step - loss: 0.6387 - accuracy: 0.7654 - val_loss: 0.5558 - val_accuracy: 0.7947
CNN Accuracy: 78.37%


In [27]:
model.save('cones_modelreal_small.keras')